In [111]:
import requests
import re
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd

In [2]:
start_url = 'https://www.tripadvisor.com/Attractions-g186534-Activities-Glasgow_Scotland.html'

In [81]:
url_container = []
prepare_url_parts = start_url.split('-Activities-')
for page in range(1,4):
    url = prepare_url_parts[0]+'-Activities-'+'oa{}-'.format(30*page)+prepare_url_parts[1]
    url_container.append(url)

In [82]:
print(url_container)

['https://www.tripadvisor.com/Attractions-g186534-Activities-oa30-Glasgow_Scotland.html', 'https://www.tripadvisor.com/Attractions-g186534-Activities-oa60-Glasgow_Scotland.html', 'https://www.tripadvisor.com/Attractions-g186534-Activities-oa90-Glasgow_Scotland.html']


In [106]:
number_list = []
name_list = []
image_list = []
score_list = []
volunme_list = []
detail_list = []

In [108]:
first_url = 'https://www.tripadvisor.com/Attractions-g186534-Activities-a_allAttractions.true-Glasgow_Scotland.html'
resp = requests.get(first_url)
bsObj = BeautifulSoup(resp.text, 'html.parser')
attraction_list = bsObj.findAll('section', {'data-automation':'AppPresentation_SingleFlexCardSection'})
for attrac in attraction_list:
    number = attrac.contents[0].span.div.div.contents[1].div.div.div.a.h3.div.span.div.get_text().split('.')[0]
    number_list.append(number)
    sitename = attrac.contents[0].span.div.div.contents[1].div.div.div.a.h3.div.span.div.get_text().split('.')[-1]
    name_list.append(sitename)
    imagelink = attrac.contents[0].span.div.div.div.div.div.div.div.a.div.div.div.div.div.div.img.attrs['src']
    image_list.append(imagelink)
    score = attrac.contents[0].span.div.div.contents[1].div.div.div.contents[1].div.svg.attrs['aria-label'].split('of')[0]
    score_list.append(score)
    review_volunme = attrac.contents[0].span.div.div.contents[1].div.div.div.contents[1].div.span.get_text()
    volunme_list.append(review_volunme)
    detail = attrac.contents[0].span.div.div.contents[1].contents[1].div.div.div.get_text()
    detail_list.append(detail)

In [109]:
for page_url in url_container:
    resp = requests.get(page_url)
    bsObj = BeautifulSoup(resp.text, 'html.parser')
    attraction_list = bsObj.findAll('section', {'data-automation':'AppPresentation_SingleFlexCardSection'})
    for attrac in attraction_list:
        number = attrac.contents[0].span.div.div.contents[1].div.div.div.a.h3.div.span.div.get_text().split('.')[0]
        number_list.append(number)
        sitename = attrac.contents[0].span.div.div.contents[1].div.div.div.a.h3.div.span.div.get_text().split('.')[-1]
        name_list.append(sitename)
        imagelink = attrac.contents[0].span.div.div.div.div.div.div.div.a.div.div.div.div.div.div.div.img.attrs['src']
        image_list.append(imagelink)
        score = attrac.contents[0].span.div.div.contents[1].div.div.div.contents[1].div.svg.attrs['aria-label'].split('of')[0]
        score_list.append(score)
        review_volunme = attrac.contents[0].span.div.div.contents[1].div.div.div.contents[1].div.span.get_text()
        volunme_list.append(review_volunme)
        detail = attrac.contents[0].span.div.div.contents[1].contents[1].div.div.div.get_text()
        detail_list.append(detail)

In [132]:
top_120 = pd.DataFrame(all_list).T

In [133]:
top_120.columns = ['number','name','image_link','score','review_volunme','detail']

In [134]:
top_120.shape

(120, 6)

In [135]:
top_120.describe()

,number,name,image_link,score,review_volunme,detail
count,120,120,120,120,120,120
unique,120,120,120,6,104,96
top,1,Glasgow Central Mosque,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,48,TheatersCity Centre
freq,1,1,1,65,3,5


In [136]:
top_120.head()

,number,name,image_link,score,review_volunme,detail
0,1,Kelvingrove Art Gallery and Museum,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"15,299",Art Museums
1,2,The Riverside Museum of Transport and Travel,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"9,436",Speciality MuseumsWest End
2,3,Glengoyne Distillery,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"1,562",Distilleries
3,4,Celtic Park,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"1,550",Arenas & Stadiums
4,5,University of Glasgow,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"1,995",Educational sitesWest End


In [137]:
top_120.isnull().sum().sort_values(ascending=False)

detail            0
review_volunme    0
score             0
image_link        0
name              0
number            0
dtype: int64

In [165]:
top_100 = top_120[:100]

In [167]:
import urllib.parse

In [168]:
for location in top_100['name']:
    address = location + ', glasgow'
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    print(response)

[{'place_id': 96555166, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 26605473, 'boundingbox': ['55.8679676', '55.8691987', '-4.2915201', '-4.2897375'], 'lat': '55.8685825', 'lon': '-4.290627759615381', 'display_name': 'Kelvingrove Art Gallery and Museum, Argyle Street, Yorkhill, Kelvinhaugh, Glasgow, Glasgow City, Scotland, G3 8AG, United Kingdom', 'class': 'tourism', 'type': 'museum', 'importance': 1.0171584767702773, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//tourist_museum.p.20.png'}]
[]
[]
[{'place_id': 256985830, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 904494, 'boundingbox': ['55.8487156', '55.8507742', '-4.2073019', '-4.2035296'], 'lat': '55.8497578', 'lon': '-4.205468869948476', 'display_name': 'Celtic Park, Kinloch Street, Barrowfield, Parkhead, Glasgow, Glasgow City, Scotland, G40 3RE, United Kingdom', 'class': 'leis

Since the latitude and longitude of all locations cannot be obtained through the Open Street Map dataset, the missing part is filled in by manual query and saved as 'lat_lon.csv'

In [164]:
lat_lon = pd.read_csv('lat_lon.csv')

In [169]:
top_100 = pd.merge(top_100,lat_lon,on=['name'])

In [170]:
top_100.head()

,number,name,image_link,score,review_volunme,detail,lat,lon
0,1,Kelvingrove Art Gallery and Museum,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"15,299",Art Museums,55.868582,-4.290628
1,2,The Riverside Museum of Transport and Travel,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"9,436",Speciality MuseumsWest End,55.865175,-4.306249
2,3,Glengoyne Distillery,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"1,562",Distilleries,56.014053,-4.363973
3,4,Celtic Park,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"1,550",Arenas & Stadiums,55.849758,-4.205469
4,5,University of Glasgow,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5,"1,995",Educational sitesWest End,55.872315,-4.289219


In [171]:
top_100.to_csv('top100.csv')

Get cover page iamge according to URL

In [172]:
path = os.getcwd() + '\\top100image\\'

In [174]:
for i in range(len(top_100['image_link'])):
    r = requests.request('get',top_100['image_link'][i]) 
    print(r.status_code)
    with open(path+str(i+1)+'.jpg','wb') as f:  
        f.write(r.content)
    f.close()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
